In [1]:
%pip install -qU langchain-groq

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [63]:
%pip install -qU langchain_community beautifulsoup4

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [26]:
%pip install docx
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

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 54 kB 161 kB/s eta 0:00:01
     |████████████████████████████████| 8.1 MB 3.3 MB/s eta 0:00:01
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53924 sha256=96a9655a8c8e7444f36be6336db912165df51ec07609bc23b3f9dce8d9f3a5d2
  Stored in directory: /Users/shivamagharkar/Library/Caches/pip/wheels/0f/8e/9d/7003eed35a84cf960876aae6bdf60d02041ddfcca66eceee94
Successfully built docx
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install tf-keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 1.5 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.2/236.2 MB 7.3 MB/s eta 0:00:0000:0100:01
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 18.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 24.3.7
    Uninstalling flatbuffers-24.3.7:
      Successfully uninstalled flatbuffers-24.3.7
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
  Attempting uninstall: keras
    Found existing installation: keras 3.1.1
    Uninstalling keras-3.1.1:
      Successfully uninstalled keras-3.1.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2

In [13]:
import fitz  # PyMuPDF
import pandas as pd
import json
from io import StringIO
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
import spacy
from sentence_transformers import SentenceTransformer, util

# Load pre-trained language model
nlp = spacy.load("en_core_web_md")  # For semantic similarity
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # For sentence embeddings

# Extract PDF data into a DataFrame
def extract_pdf_data(pdf_path):
    skills = []
    experience = []
    education = []

    try:
        doc = fitz.open(pdf_path)  # Open the PDF document
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)  # Load each page
            text = page.get_text()
            lines = text.split('\n')
            
            is_skills_section = is_experience_section = is_education_section = False
            
            for line in lines:
                # Handle section starts
                if 'Skills' in line or 'Technical Skills' in line:
                    is_skills_section = True
                    is_experience_section = is_education_section = False
                    continue
                elif 'Experience' in line:
                    is_experience_section = True
                    is_skills_section = is_education_section = False
                    continue
                elif 'Education' in line:
                    is_education_section = True
                    is_skills_section = is_experience_section = False
                    continue
                
                # Extract data within respective sections
                if is_skills_section and line.strip():
                    skills.extend([skill.strip() for skill in line.split(',') if skill.strip()])
                elif is_experience_section and line.strip():
                    experience.append(line.strip())
                elif is_education_section and line.strip():
                    education.append(line.strip())
        
        doc.close()  # Close the document
        
        # Convert to DataFrame
        data = {
            "skills": skills if skills else ["No skills found"],
            "experience": experience if experience else ["No experience found"],
            "education": education if education else ["No education found"]
        }
        return pd.DataFrame([data])
    
    except Exception as e:
        print(f"Error extracting PDF data: {e}")
        return pd.DataFrame()

# Load resume and extract data
pdf_resume_path = "/Users/shivamagharkar/Desktop/Resumes/SHIVAM_AGHARKAR_-_CV.pdf"  # Update this path
resume_df = extract_pdf_data(pdf_resume_path)
print("Resume Data in DataFrame:\n", resume_df)

# Convert DataFrame to JSON
resume_json = resume_df.to_json(orient="records")
resume_json_dict = pd.read_json(StringIO(resume_json))

# Scrape the job data using WebBaseLoader
loader = WebBaseLoader("https://jobs.lever.co/justwatch/27468019-6219-4007-a69f-0453441c96f4?lever-source=LinkedIn&source=6")
page_data = loader.load().pop().page_content

# LLM setup for job extraction
llm = ChatGroq(
    temperature=0,
    groq_api_key='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',  # Replace with your actual API key
    model_name="llama-3.1-70b-versatile"
)

# Define a job extraction prompt
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    Extract job postings from the website and return them in JSON format with the following keys: 
    'role', 'experience', 'skills', and 'description'.
    Return only valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

# Extract job data using LLM with error handling
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data': page_data})

# Print raw response for debugging
print("Raw LLM response:\n", res.content)

# Initialize job_data
job_data = None

# Check if the LLM returned valid JSON
try:
    if isinstance(res.content, bytes):
        response_content = res.content.decode('utf-8')  # Convert bytes to string
    else:
        response_content = res.content

    response_content = response_content.strip('```json').strip('```')

    job_data = json.loads(response_content)
    print("Extracted Job Data:", job_data)
except json.JSONDecodeError as e:
    print("JSONDecodeError occurred:", e)
    print("Raw response content is not valid JSON:\n", response_content)
except Exception as e:
    print("An unexpected error occurred:", e)
    print("Raw response content:\n", response_content)

# Check if job_data was successfully defined
if job_data is None:
    print("Job data could not be extracted. Please check the response format.")
else:
    # Extract and compare skills from resume and job description
    student_skills = resume_json_dict['skills'].iloc[0]
    job_skills = job_data.get('skills', [])

    # Convert skills into embeddings using SentenceTransformer
    student_skill_embeddings = model.encode(student_skills)
    job_skill_embeddings = model.encode(job_skills)

    # Find the cosine similarity between each skill from resume and job description
    similarities = util.cos_sim(student_skill_embeddings, job_skill_embeddings)

    # Set a threshold for considering skills as matched
    similarity_threshold = 0.7

    # Extract matches based on the similarity threshold
    matched_skills = set()
    for i, student_skill in enumerate(student_skills):
        for j, job_skill in enumerate(job_skills):
            similarity_score = similarities[i][j]
            if similarity_score > similarity_threshold:
                matched_skills.add(job_skill)  # Or add both if you want exact pairs

    print(f"Matched Skills: {matched_skills}")

    # Define a prompt for cold email generation
    prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        Write a cold email mentioning the matched skills: {matched_skills}.
        
        ### EMAIL (NO PREAMBLE):
        """
    )

    # Generate the cold email using LLM
    chain_email = prompt_email | llm
    email_response = chain_email.invoke({
        "job_description": str(job_data),
        "matched_skills": ', '.join(matched_skills)
    })

    # Print the generated cold email
    print("Generated Cold Email:\n", email_response.content)


/Users/shivamagharkar/miniconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Error extracting PDF data: module 'fitz' has no attribute 'open'
Resume Data in DataFrame:
 Empty DataFrame
Columns: []
Index: []
Raw LLM response:
 ```json
{
  "role": "ML & Data Science Engineer",
  "experience": "Not specified",
  "skills": [
    "Machine Learning",
    "Data Science",
    "Python",
    "Pytorch",
    "Scipy",
    "XGBoost",
    "Snowflake",
    "NLP",
    "Tabular modeling",
    "Forecasting",
    "Normalization"
  ],
  "description": "We are looking for a builder who is fearless in challenging conventions and bringing new ideas to the table. As an ML & Data Science Engineer, you will own the machine room of the data products driving our business, improve our automated campaign steering, and advance our targeting by extracting every ounce of entertainment intelligence from our recommendation system based on massive user and item data."
}
```
Extracted Job Data: {'role': 'ML & Data Science Engineer', 'experience': 'Not specified', 'skills': ['Machine Learning', 'Dat

KeyError: 'skills'

In [ ]:
import fitz  # PyMuPDF
import pandas as pd
import json
from io import StringIO
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from sentence_transformers import SentenceTransformer, util

# Load pre-trained language model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # For sentence embeddings

# Function to extract PDF data into a DataFrame
def extract_pdf_data(pdf_path):
    skills = []
    experience = []
    education = []

    try:
        doc = fitz.open(pdf_path)  # Open the PDF document
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)  # Load each page
            text = page.get_text()
            lines = text.split('\n')
            
            is_skills_section = is_experience_section = is_education_section = False
            
            for line in lines:
                # Handle section starts
                if 'Skills' in line or 'Technical Skills' in line:
                    is_skills_section = True
                    is_experience_section = is_education_section = False
                    continue
                elif 'Experience' in line:
                    is_experience_section = True
                    is_skills_section = is_education_section = False
                    continue
                elif 'Education' in line:
                    is_education_section = True
                    is_skills_section = is_experience_section = False
                    continue
                
                # Extract data within respective sections
                if is_skills_section and line.strip():
                    skills.extend([skill.strip() for skill in line.split(',') if skill.strip()])
                elif is_experience_section and line.strip():
                    experience.append(line.strip())
                elif is_education_section and line.strip():
                    education.append(line.strip())
        
        doc.close()  # Close the document
        
        # Convert to DataFrame
        data = {
            "skills": skills if skills else ["No skills found"],
            "experience": experience if experience else ["No experience found"],
            "education": education if education else ["No education found"]
        }
        return pd.DataFrame([data])
    
    except Exception as e:
        print(f"Error extracting PDF data: {e}")
        return pd.DataFrame()

# Load resume and extract data
pdf_resume_path = "/content/SHIVAM_AGHARKAR_-_CV.pdf"  # Update with actual path
resume_df = extract_pdf_data(pdf_resume_path)
student_skills = [skill.lower().strip() for skill in resume_df['skills'].iloc[0]]

# Load job data using WebBaseLoader
loader = WebBaseLoader("https://jobs.lever.co/justwatch/27468019-6219-4007-a69f-0453441c96f4?lever-source=LinkedIn&source=6")  # Update with actual URL
page_data = loader.load().pop().page_content

# LLM setup for job extraction
llm = ChatGroq(
    temperature=0,
    groq_api_key='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',  # Replace with your actual API key
    model_name="llama-3.1-70b-versatile"
)

# Define a job extraction prompt
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    Extract skills from the job posting.
    Return skills in JSON format.
    """
)

# Extract job data
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data': page_data})

# Check if response is valid JSON
try:
    print("Raw LLM Response:", res.content)  # Debug: Print the raw response
    job_data = json.loads(res.content) if res.content else {}
except json.JSONDecodeError as e:
    print("JSONDecodeError:", e)
    print("Raw response content is not valid JSON:\n", res.content)
    job_data = {}

# Extract job skills
job_skills = [skill.lower().strip() for skill in job_data.get('skills', [])]

# Print extracted skills for debugging
print("Student Skills:", student_skills)
print("Job Skills:", job_skills)

# Check if job_skills is empty before proceeding
if not job_skills:
    print("No job skills extracted. Cannot compute similarity.")
else:
    # Compute embeddings and similarities
    student_skill_embeddings = model.encode(student_skills)
    job_skill_embeddings = model.encode(job_skills)
    
    # Ensure both embeddings are not empty before computing similarities
    if student_skill_embeddings.size == 0 or job_skill_embeddings.size == 0:
        print("One of the skill sets has no embeddings. Cannot compute similarity.")
    else:
        similarities = util.cos_sim(student_skill_embeddings, job_skill_embeddings)

        # Set a threshold for considering skills as matched
        similarity_threshold = 0.10

        # Extract matches based on the similarity threshold
        matched_skills = set()
        for i, student_skill in enumerate(student_skills):
            for j, job_skill in enumerate(job_skills):
                similarity_score = similarities[i][j]
                print(f"Similarity between '{student_skill}' and '{job_skill}': {similarity_score:.4f}")
                if similarity_score > similarity_threshold:
                    matched_skills.add(job_skill)

        print(f"Matched Skills: {matched_skills}")

# Generate a cold email if there are matched skills
if matched_skills:
    prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        Write a cold email mentioning the matched skills: {matched_skills}.
        
        ### EMAIL (NO PREAMBLE):
        """
    )

    # Generate the cold email using LLM
    chain_email = prompt_email | llm
    email_response = chain_email.invoke({
        "job_description": str(job_data),
        "matched_skills": ', '.join(matched_skills)
    })

    # Print the generated cold email
    print("Generated Cold Email:\n", email_response.content)
else:
    print("No matched skills found. Please check the extraction and comparison process.")
